# Deep Research Agent - UK Global Talent Visa Eligibility Evaluator

## Project Overview
The Deep Research Agent is an intelligent system that automates the evaluation of candidate profiles against UK Global Talent Visa requirements. It combines real-time web research with AI-powered analysis to provide comprehensive, up-to-date assessments of visa eligibility, helping applicants understand their chances of success before submitting their applications.

In [47]:
# Imports
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
from pypdf import PdfReader
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown
from openai import OpenAI

In [48]:
load_dotenv(override=True)
openai = OpenAI()

In [49]:
# Schema and Classes
class WebSearchItem(BaseModel):
    reason: str = Field(
        description="Your reasoning for why this search is important to the query."
    )

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(
        description="A list of web searches to perform to best answer the query."
    )


class Resume(BaseModel):
    resume_extract: str = Field(
        description="The Extracted text from the loaded resume PDF"
    )

    resume_url: str = Field(
        description="The location of the resume pdf file. It can be wither hosted locally, or remotely"
    )


class ReportData(BaseModel):
    short_summary: str = Field(
        description="A short 2-3 sentence summary of the findings."
    )

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(
        description="Suggested topics to research further"
    )

### Read the Resume

In [50]:
def read_resume(url: Resume) -> Resume:
    reader = PdfReader(url)
    resume = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            resume += text
    return resume

### The OpenAI Search Agent

In [52]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

### Planner Agent

In [53]:
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant specializing in UK immigration. \
Given a query about the UK Global Talent Visa, come up with a set of web searches to find the most current and accurate information. \
Focus on official sources, recent policy updates, requirements, and application guidance. \
Output {HOW_MANY_SEARCHES} search terms that will help gather up-to-date information about the UK Global Talent Visa."

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

### Send Email Tool

In [54]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("tochinwachukwu33@gmail.com") # Change this to your verified email
    to_email = To("tochinwachukwu33@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return "success"

### Send Email Agent using the Tool

In [55]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)



### Report Writing Agent

In [56]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [66]:
async def plan_searches(query: str):
    """Use the planner_agent to plan which searches to run for the query"""
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output


async def perform_searches(search_plan: WebSearchPlan):
    """Call search() for each item in the search plan"""
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results


async def search(item: WebSearchItem):
    """Use the search agent to run a web search for each item in the search plan"""
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output


async def evaluate_profile(search_res, resume_text):
    name = "Tochi Nwachukwu"
    system_prompt = f"You are an expert immigration consultant evaluating a resume against Global Talent Visa requirements. \
    Your task is to analyze {name}'s resume and assess how well it aligns with the visa criteria. \
    Provide a detailed, objective evaluation highlighting strengths, potential gaps, and recommendations for improvement. \
    Be thorough, professional, and constructive in your assessment."

    system_prompt += f"\n\n## Global Talent Visa Requirements:\n{search_res}\n\n"
    system_prompt += f"## Resume to Evaluate:\n{resume_text}\n\n"

    user_message = f"Please analyze {name}'s resume against the UK Global Talent Visa requirements provided. \
    In your evaluation:\n\
    1. Identify which requirements are clearly met with supporting evidence from the resume\n\
    2. Note any requirements that are partially met or unclear\n\
    3. Highlight any gaps or missing elements\n\
    4. Provide specific, actionable recommendations to strengthen the visa application\n\
    5. Give an overall assessment of the likelihood of success"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message},
    ]

    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
# resume_link = "me/linkedin.pdf"
# resume_text = read_resume(resume_link)

# profile_evaluation = await evaluate_profile(resume_text)
# print(profile_evaluation)

### Evaluation of Tochi Nwachukwu's Resume Against UK Global Talent Visa Requirements

#### 1. Requirements Clearly Met:

- **Significant Expertise and Proven Track Record**: 
  Tochi describes over six years of hands-on experience as a DevOps Engineer, demonstrating significant expertise in architecting cloud workloads, particularly on AWS. His experience includes leading initiatives for the Federal Government of Nigeria and impactful projects in various sectors.
  
- **Professional Recognition**: 
  He has received a scholarship for academic excellence and was recognized as the Best Graduating Student in Heat and Mass Transfer, indicating his academic credentials and achievements.

- **Professional Certifications**: 
  The AWS Certified Developer Associate and AWS Certified Cloud Practitioner indicate a high level of qualification. His ongoing certification efforts (with AWS SysOps Administrator in view) and pursuit of Google Professional Cybersecurity Certification further demonstra

/var/folders/c0/3t5lpmc91xqb9f5z7rjs1z7m0000gn/T/ipykernel_13602/3242160625.py:4: RuntimeWarning: coroutine 'evaluate_profile' was never awaited
  profile_evaluation = await evaluate_profile(resume_text)


In [64]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

In [67]:
resume_link = "me/linkedin.pdf"
query = "What are the detailed eligibility requirements for the UK Global Talent Visa? \
I want to evaluate whether a candidate's background meets the criteria for endorsement."

with trace("Research trace"):
    print("Starting research...")
    resume_text = read_resume(resume_link)
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    profile_evaluation = await evaluate_profile(search_results, resume_text)
    print(profile_evaluation)
    report = await write_report(query, profile_evaluation)
    await send_email(report)
    print("Hooray!")

Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Tochi Nwachukwu's resume demonstrates a strong background aligned with aspects of the UK Global Talent Visa requirements, primarily in the Digital Technology pathway. Below is a comprehensive evaluation based on the specified criteria.

### 1. Requirements Clearly Met:

- **Exceptional Talent in Digital Technology**: 
  - Tochi has over 6 years of experience as a DevOps Engineer, which includes architecting and building scalable cloud workloads primarily on AWS. His experience with organizations like Illuminem, Schull Technologies, and TalentUp demonstrates engagement in innovative technology solutions.
  - He has led DevOps initiatives that align with the required criteria, showcasing leadership qualities and significant contributions to the digital technology sector.
  
- **Professional Certifications**:
  - Certifications such as AWS Certified Developer Associate and AWS Certified Cloud